NOMBRES Y APELLIDOS: ALEX HELDER HUANCARA CCOLQQUE \\

CURSO: MINERIA DE DATOS \\

DOCENTE: CARLOS FERNANDO MONTOYA CUBAS

In [1]:
!pip install pyspark

In [2]:
from pyspark import SparkContext
sc = SparkContext()

In [3]:
import math
from pyspark.sql.functions import *

In [7]:
def clean_data(textRDD):
  #Eliminar lineas vacias
  textRDD=textRDD.filter(lambda x : x is not '')
  tupleData = []
  i=1
  for line in textRDD.collect():
    tupla=(i,line)
    tupleData.append(tupla)
    i+=1
  return tupleData

""" Creamos los valores unicos de cada token"""
def unique(data1,particiones=4):
  linesRDD = sc.parallelize(data1,particiones)
  parTokenRDD = linesRDD.flatMap(lambda x: [((x[0],i.lower()),1) for i in x[1].split()])
  return parTokenRDD

""" Creamos la bolsa de palabras"""
def bow(bagOfWordsRDD):
  #obtenemos la frecuencia de cada palabra por documento
  frecPerDocumentRDD=bagOfWordsRDD.reduceByKey(lambda x,y:x+y)
  return frecPerDocumentRDD

"""Hallando el TF"""
def TF(bagOfWordsRDD):
  frecuenciasRDD=bow(bagOfWordsRDD)
  # Mapear el RDD para obetener ('token', ('id',frecuencia))
  tfRDD=frecuenciasRDD.map(lambda x: (x[0][1],(x[0][0],x[1])))
  return tfRDD

"""Hallando el IDF"""
def IDF(bagOfWordsRDD,cantidadDoc):
  frecuenciasRDD=bow(bagOfWordsRDD)
  # Mapeamos para colocar cada token como valor clave
  tokenRDD=frecuenciasRDD.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
  # Extraemos el token y su ocurrencia por documento
  tokenOcurrenciaDocRDD=tokenRDD.map(lambda x:(x[0],x[1][2]))
  # contamos la contidad de documentos que contienen un token
  docPerTokenRDD=tokenOcurrenciaDocRDD.reduceByKey(lambda x,y:x+y)
  #calculamos el logaritmo para el IDF
  idfRDD = docPerTokenRDD.map(lambda x: (x[0],math.log10(cantidadDoc/x[1])))
  return idfRDD

"""juntamos el TF de las frecuencias y los valores de IDF"""
def TF_IDF(bagOfWordsRDD,cantidadDoc):
  # valores tf
  tfRDD = TF(bagOfWordsRDD)
  # puntuaciones idf
  idfRDD = IDF(bagOfWordsRDD,cantidadDoc)
  # unimos los valores de TF e IDF
  tfIdfRDD = tfRDD.join(idfRDD)
  # tf * idf (multiplicamos los valores)
  tfIdfRDD = tfIdfRDD.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()
  # organizacion para mostrado de datos
  tfIdfRDD = tfIdfRDD.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))
  return tfIdfRDD

In [8]:
# Prueba
twittsRDD = sc.textFile('twitts.txt',4)
twittsRDD = clean_data(twittsRDD)
num_docs = len(twittsRDD)
bag_of_words = unique(twittsRDD, )
tf_idf = TF_IDF(bag_of_words, num_docs)

# Mostrar los valores de tf-idf
tf_idf.collect()

[(1, 'store', 1, 1.105595616949241, 1.105595616949241),
 (1, 'diein', 1, 1.399423217328898, 1.399423217328898),
 (1, 'retweet', 1, 1.399423217328898, 1.399423217328898),
 (1, 'agree', 1, 1.3690792008697128, 1.3690792008697128),
 (1, 'stage', 1, 1.3798696809532192, 1.3798696809532192),
 (1, 'nyc', 1, 1.353380100659997, 1.353380100659997),
 (1, 'protester', 1, 1.388142206919209, 1.388142206919209),
 (1, 'protest', 1, 1.377146822617746, 1.377146822617746),
 (1, 'anger', 1, 1.388142206919209, 1.388142206919209),
 (1, 'misplace', 1, 1.390935107103379, 1.390935107103379),
 (2, 'range', 1, 2.2195389692560776, 2.2195389692560776),
 (2, 'trading', 1, 1.928054291498327, 1.928054291498327),
 (2, 'stock', 1, 1.457415164306585, 1.457415164306585),
 (2, 'mkt', 1, 1.968482948553935, 1.968482948553935),
 (2, 'update', 1, 1.3937460844372076, 1.3937460844372076),
 (2, 'apple', 1, 0.578668824553762, 0.578668824553762),
 (2, 'intraday', 1, 1.6270242958343457, 1.6270242958343457),
 (2, 'premium', 1, 2.0761